動的にネットワークを構築する書き方。複数種類のネットワークを構築するために構造をyaml形式の設定ファイル(config.yaml)に書き、読み込みインスタンスを作成する。

In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
from dgl.nn import GraphConv,MaxPooling
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.optim as optim
import numpy as np
import time
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import os
import yaml


%matplotlib inline

In [5]:
#動的にネットワークを構築。nn.ModuleListで中間層をリスト形式で構築。
class DynamicNet(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(DynamicNet, self).__init__()
        # ネットワーク内のパラメータを定義します。
        self.input_layer = nn.Linear(input_size, hidden_sizes[0])
        self.middle_layers = nn.ModuleList([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]) for i in range(len(hidden_sizes)-1)])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x):
        # forward関数を定義します。この関数は、xを入力として受け取り、出力yを返します。
        h_relu = self.input_layer(x).clamp(min=0)
        for layer in self.middle_layers:
            h_relu = layer(h_relu).clamp(min=0)
        y_pred = self.output_layer(h_relu)
        return y_pred


In [6]:
#設定ファイル(config.yaml)を読み込む
with open('./garbage/config.yaml','r') as f:
    config=yaml.safe_load(f)

nets=[]
#configから設定ごとに各数値を読み込み、ネットワークを構築し、netsに格納する。
for model in config:
    input_size=config[model]['input_size']
    hidden_size = config[model]['hidden_size']
    output_size = config[model]['output_size']

    net=DynamicNet(input_size,hidden_size,output_size)
    nets.append(net)

nets

[DynamicNet(
   (input_layer): Linear(in_features=81, out_features=60, bias=True)
   (middle_layers): ModuleList(
     (0): Linear(in_features=60, out_features=40, bias=True)
     (1): Linear(in_features=40, out_features=30, bias=True)
   )
   (output_layer): Linear(in_features=30, out_features=10, bias=True)
 ),
 DynamicNet(
   (input_layer): Linear(in_features=81, out_features=200, bias=True)
   (middle_layers): ModuleList(
     (0): Linear(in_features=200, out_features=100, bias=True)
     (1): Linear(in_features=100, out_features=50, bias=True)
   )
   (output_layer): Linear(in_features=50, out_features=10, bias=True)
 )]

In [7]:
print(hidden_size)

[200, 100, 50]


In [6]:
input_size, hidden_sizes, output_size = 10, [20, 30, 40], 5
net = DynamicNet(input_size, hidden_sizes, output_size)

In [9]:
#ダミーの結果をyaml形式で保存するコード
log={"loss":[0.5,0.4,0.3,0.2,0.1],
     "acc":[0.8,0.85,0.9,0.95,0.98]}
filename='test_log.yaml'
with open(filename,"w") as f:
    yaml.dump(log,f,default_flow_style=False)